In [2]:
from scipy.spatial import distance
from skimage import data
from skimage import io, color, img_as_ubyte, util
import warnings
warnings.filterwarnings('ignore')
from skimage import io, img_as_ubyte
from skimage.color import rgb2gray
import cv2
import numpy as np
import matplotlib.pyplot as plt
t=0.0000000000000000000000000001
import matplotlib.patches as mpatches
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb
import math
from skimage import filters
##########################################################################################################

def show2imgs(im1, im2, title1='Obraz pierwszy', title2='Obraz drugi', size=(10,10)):
    
    import matplotlib.pyplot as plt
    
    f, (ax1, ax2) = plt.subplots(1,2, figsize=size)
    ax1.imshow(im1, cmap='gray')
    ax1.axis('off')
    ax1.set_title(title1)

    ax2.imshow(im2, cmap='gray')
    ax2.axis('off')
    ax2.set_title(title2)
    plt.show()

###Współczynnik Fereta###
def computeFeret(points):
    
    px = [x for (y,x) in points]
    py = [y for (y,x) in points]
    
    fx = max(px) - min(px)
    fy = max(py) - min(py)
    
    return float(fy)/(float(fx)+t)  

###Współczynnik Blair-Bliss####

def computeBB(points):
    
    s = len(points)
    my,mx = cog2(pts)
    
    r = 0
    for point in points:
         r = r + distance.euclidean(point,(my,mx))**2
            
    return s/(math.sqrt(2*math.pi*r)+t)

def cog2(points):
    mx=0
    my=0
    for (y,x) in points:
        mx = mx + x
        my = my + y
    mx = mx/len(points)
    my = my/len(points)
    
    return [my, mx]

###srodek cięzkości###
def cog(labelledImage, objNumber):
    cx = 0
    cy = 0
    for y in range(labelledImage.shape[0]):
        for x in range(labelledImage.shape[1]):
            if labelledImage[y,x] == objNumber:
                cy = cy + y
                cx = cx + x

    cx = cx / sizes[objNumber]
    cy = cy / sizes[objNumber]

    return int (cy), int (cx)

def getFigure(labelledImage, objNumber):
    
    points = []
    for y in range(labelledImage.shape[0]):
        for x in range(labelledImage.shape[1]):
            if labelledImage[y,x] == objNumber:
                points.append((y,x))

    return points





ModuleNotFoundError: No module named 'cv2'

In [ ]:
im="10.jpg"
obr = io.imread(im)
obr = color.rgb2gray(obr)
obr = img_as_ubyte(obr) 
show2imgs(obr,obr,title1="1",title2="2",size=(25,25))


In [ ]:

th = 150
th, bim = cv2.threshold(obr, thresh=th, maxval=255, type=cv2.THRESH_OTSU)

show2imgs(obr, bim, title1='Obraz oryginalny', title2='Obraz po binaryzacji', size=(25,25))

In [ ]:

kernel = np.ones((8,8),np.uint8)
erodeBin = cv2.erode(bim, kernel=kernel, iterations=2)
show2imgs(bim, erodeBin, title1='Obraz oryginalny', title2='Obraz erozji', size=(20,20))

In [ ]:
image = erodeBin[5:-5, 5:-5]
ilosc = 0

# OTSU
thresh = threshold_otsu(image)
bw = closing(image > thresh, square(3))

#usuwa zakłócenia na granicy
cleared = clear_border(bw)

label_image = label(cleared)
image_label_overlay = label2rgb(label_image, image=image)

fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(image_label_overlay)

for region in regionprops(label_image):
    if region.area >= 20:
        ilosc+=1
        #rysuje kwadraty opisane na obiektach
        minr, minc, maxr, maxc = region.bbox
        
        rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                                  fill=False, edgecolor='yellow', linewidth=1)
        ax.add_patch(rect)

ax.set_axis_off()
plt.tight_layout()
plt.show() 

print('ziaren jest:', ilosc)

In [ ]:
image = util.invert(erodeBin)
image = rgb2gray(image)

In [ ]:
th = 1

print('Prog binaryzacji: ', th)

binary = image < th

plt.imshow(binary, cmap='binary')
plt.axis('on')
plt.suptitle('Obraz po binaryzacji')
plt.show()

In [ ]:
from scipy import ndimage as ndi
label_objects, nb_labels = ndi.label(binary)

sizes = np.bincount(label_objects.ravel())
mask_sizes = sizes > 20
mask_sizes[0] = 0

figures = mask_sizes[label_objects]

In [ ]:
plt.imshow(figures, cmap="hot")
plt.axis('on')
plt.suptitle('Obiekty na obrazie')
plt.show()

In [ ]:
cogs = []
for i in range(nb_labels):
    cogs.append(cog(label_objects, i+1))
print('Środki ciężkości dla obiektów')
print(cogs)

In [ ]:
print (nb_labels)

In [ ]:

for i in range(nb_labels):
    pts = getFigure(label_objects, i+1)
    bb = computeBB(pts)
    feret = computeFeret(pts)
    
    print('Liczba punktow: ',len(pts),'\nBlair-Bliss: ', bb,'\nFeret: ',feret, '\n---\n')